# Changes in firing regularity in the normal and impaired auditory system

**Dan F. M. Goodman, Ian M. Winter, Agnès C. Léger, Alain de Cheveigné, Christian Lorenzi**

See the preprint of the paper [here](http://biorxiv.org/content/early/2017/03/28/121707).

The interactive paper has six notebooks, providing interactive versions of some of the figures in the paper. The notebooks are listed below:

1. [Basic model](basic_model.ipynb)
2. [Behaviour maps](maps.ipynb)
3. [Level dependence](level_dependence_density.ipynb)
4. [Deafferentation](deafferentation.ipynb)
5. [The effect of deafferentation on modulation transfer function](amplitude_modulation.ipynb)
6. [Analytical solution to basic model](analytical_solution.ipynb)